In [9]:
# Establecemos los directorios del proyecto
import sys
from pathlib import Path
sys.path.append(str(Path('.').resolve().parent))

In [2]:
from langchain import PromptTemplate
# from langchain_community.chat_models import ChatOpenAI
import base64
import os
from api.configuration.keys import POKER_GPT_KEY
from langchain_openai import ChatOpenAI
#from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from api.configuration.config import IMG_DIR


/home/y41000/repos/gpt-poker-player/api/images


In [3]:

# Almacenamos modelo
gptmodel = ChatOpenAI(model_name = "gpt-4o", openai_api_key=POKER_GPT_KEY)
# Mi username en la plataforma de poker
#user_poker_name="pokeados"

In [4]:
print(gptmodel.invoke('Hola').content)
print(gptmodel([HumanMessage(content="Hola")]).content)

¡Hola! ¿Cómo puedo ayudarte hoy?


/home/y41000/miniconda3/envs/poker/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


¡Hola! ¿Cómo estás? ¿En qué puedo ayudarte hoy?


CONSTRUIMOS EL TEMPLATE

In [66]:
# Función para codificar la imagen que le vamos a pasar a langchain

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


In [67]:
context_temp = """You are a profesional poker player called {user_poker_name}.
Current situation: We are in {stage}. Player {player} make a {play} from {seat} position.
Players statisticts: Player {player} have played {n_plays} hands with {n_raises} rises and {n_wins} wins.
See the table: {imagen}
Tell me: What move would you do and why."""

imagen = encode_image(f"{IMG_DIR}/Mano_2024-05-12:23-14-41.png")

# construimos el template
template = PromptTemplate(
    input_variables=['user_poker_name','stage','player','play','seat','player','n_plays','n_raises','n_wins','imagen'],
    template = context_temp)

# Vemos cómo sale el prompt
prompt_test = template.format(user_poker_name='pokeados',
                              player='HER_MILK_MAN',
                              stage='preflop',
                              seat='UTG',
                              play = 'call',
                              n_plays = 33,
                              n_raises = 25,
                              n_wins = 2,imagen=imagen)
#print(prompt_test)

Así a lo bruto no es posible procesar la información porque la imagen arrasa con el límite de tokens.
Vamos a usar las cadenas de LangChain.

In [62]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import matplotlib
matplotlib.use('TkAgg')  # Usar la backend TkAgg, que es interactiva
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import polars as pl

import base64
import os
import ast
from api.configuration.keys import POKER_GPT_KEY
from api.configuration.config import IMG_DIR,TAB_DIR

import re


In [11]:
# Función para codificar la imagen
def encode_image(image_path):
    with open(image_path, 'rb') as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')
    
# Modelo
llm = ChatOpenAI(model_name="gpt-4o", openai_api_key=POKER_GPT_KEY)


In [56]:
# PRIMERA CADENA

# En la primera cadena vamos a configurar la imagen que le vamos a enviar a la segunda cadena
# Imagen
image_path = f"{IMG_DIR}/Mano_2024-05-12:23-13-25.png"
base64_image = encode_image(image_path)
# Componente LangChain
infromacion_mesa = HumanMessage(
    content=[
        {
            "type": "text", 
            "text": """
                Extract all the relevant information of the table, like players, pot, chips of every player, situation of the table, etc. After that, provide a dictionary called player_info with the following structure, with no comments neither '\\n' character :
                {
                    'player_name1': {
                        'game_phase': just one of ('preflop', 'flop', 'turn', 'river'),
                        'table_position': just one of ('BTN', 'SB', 'BB', 'UTG', 'MP', 'HJ')
                    },
                    ...
                    'player_nameN': {
                        'game_phase': just one of ('preflop', 'flop', 'turn', 'river'),
                        'table_position': just one of ('BTN', 'SB', 'BB', 'UTG', 'MP', 'HJ')
                    }
                }
            """
        },
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        }
    ]
)

# Ejecutar el modelo para obtener el contenido del prompt
response = llm.invoke([infromacion_mesa]).content

# Dividimos la salida. El diccionario se usará para buscar las estadísticas de los jugadores, y el resto para la cadena.
dic_players = ast.literal_eval((re.search(r'player_info = ({.+})', str(response),re.DOTALL).group(1).replace("\n","")))
response = re.search(r'(.+)player_info = ', str(response),re.DOTALL).group(1)

# Crear el PromptTemplate a partir de la respuesta
infromacion_mesa_template = PromptTemplate.from_template(response)


# Creamos la cadena de LangChain con el LLM y el prompt/imagen de la mesa de poker
chain1 = LLMChain(llm=llm, prompt=infromacion_mesa_template, output_key="Current_situation") # Estos dos parámetros son los más importantes para la cadena





In [116]:
# PRIMERA CADENA

# En la primera cadena vamos a configurar la imagen que le vamos a enviar a la segunda cadena
# Imagen
image_path = f"{IMG_DIR}/Mano_2024-05-12:23-13-25.png"
base64_image = encode_image(image_path)
# Componente LangChain
infromacion_mesa = HumanMessage(
    content=[
        {
            "type": "text", 
            "text": """
                First extract and show all the relevant information of the table, like players, pot, chips of every player, situation of the table, etc. Take into account that the button is always the player with the token in the table with the spades simbol. After that, ignoring pokeados player, provide a dictionary called player_info (it means, write player_info = {....}), to use ast.literal_eval over it, with the following structure, with no comments neither '\\n' character :
                {
                    'player': [player_name1, player_name2, ..., player_nameN],
                    'stage': [stage of player_name1, stage of player_name2, ..., stage of player_nameN] (just one of 'preflop', 'flop', 'turn', 'river')
                    'seat': [seat of player_name1, seat of player_name2, ..., seat of player_nameN] (just one of 'BTN', 'SB', 'BB', 'UTG', 'MP', 'HJ')
                }
            """
        },
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        }
    ]
)

# Ejecutar el modelo para obtener el contenido del prompt
response = llm.invoke([infromacion_mesa]).content

# Dividimos la salida. El diccionario se usará para buscar las estadísticas de los jugadores, y el resto para la cadena.
dic_players = ast.literal_eval((re.search(r'player_info = ({.+})', str(response),re.DOTALL).group(1).replace("\n","")))
response2 = re.search(r'(.+)player_info = ', str(response),re.DOTALL).group(1)

# Crear el PromptTemplate a partir de la respuesta
infromacion_mesa_template = PromptTemplate.from_template(response2)


# Creamos la cadena de LangChain con el LLM y el prompt/imagen de la mesa de poker
chain1 = LLMChain(llm=llm, prompt=infromacion_mesa_template, output_key="Current_situation") # Estos dos parámetros son los más importantes para la cadena





In [117]:
print(response)

### Extracted Information:

**Players and Chips:**
- pokeados: 2,384 chips
- isergiu1992: 5,000 chips
- ida_bless: 19,450 chips
- Chi Long Quaw: 9,200 chips
- carla.1985rm: 10,000 chips
- dudatpa: 31,877 chips
- marklt19: 8,284 chips
- HER_MILK_MAN: 9,105 chips

**Pot:**
- 250 chips

**Player Positions:**
- **Button (BTN):** carla.1985rm
- **Small Blind (SB):** dudatpa
- **Big Blind (BB):** marklt19
- **Under the Gun (UTG):** HER_MILK_MAN
- **Middle Position (MP):** pokeados (to be ignored)
- **Hijack (HJ):** isergiu1992
- **Cutoff (CO):** ida_bless

**Stage of the Game:**
- Preflop (as no community cards are shown yet)

### Dictionary:
```python
player_info = {
    'player': ['isergiu1992', 'ida_bless', 'Chi Long Quaw', 'carla.1985rm', 'dudatpa', 'marklt19', 'HER_MILK_MAN'],
    'stage': ['preflop', 'preflop', 'preflop', 'preflop', 'preflop', 'preflop', 'preflop'],
    'seat': ['HJ', 'CO', 'MP', 'BTN', 'SB', 'BB', 'UTG']
}
```


In [118]:
pl.DataFrame(dic_players)

player,stage,seat
str,str,str
"""isergiu1992""","""preflop""","""HJ"""
"""ida_bless""","""preflop""","""CO"""
"""Chi Long Quaw""","""preflop""","""MP"""
"""carla.1985rm""","""preflop""","""BTN"""
"""dudatpa""","""preflop""","""SB"""
"""marklt19""","""preflop""","""BB"""
"""HER_MILK_MAN""","""preflop""","""UTG"""


In [119]:
# EXTRACCIÓN DE ESTADÍSTICA DE JUGADORES
by_stage = pl.read_parquet(f'{TAB_DIR}/by_stage.parquet')
stages_reached = pl.read_parquet(f'{TAB_DIR}/stages_reached.parquet')

by_stage=by_stage.join(pl.DataFrame(dic_players)
    , on=['player', 'seat','stage'], how="inner", coalesce=True)

stages_reached=stages_reached.join(pl.DataFrame(dic_players)
    , on=['player', 'seat'], how="inner", coalesce=True)


In [121]:
stages_reached

player,max_stage,seat,stage_reached,hands_won,hands_lost,hands_null,hands_folds,n_checks,n_calls,n_bets,n_raises,stage
str,str,str,u32,i32,i32,i32,i32,i32,i32,i32,i32,str


In [122]:
# SEGUNDA CADENA

# construimos el template donde le damos el contexto de los jugadores

context_temp = """You are a profesional poker player called {user_poker_name}.
Current situation: We are in {stage}. Player {player} make a {play} from {seat} position.
Players statisticts: Player {player} have played {n_plays} hands with {n_raises} rises and {n_wins} wins.
This is the current situation of the table:{Current_situation}
Tell me: What move would you do. Just tell me the play, and why, in two diferents bullets."""

template = PromptTemplate(
    input_variables=['user_poker_name','stage','player','play','seat','player','n_plays','n_raises','n_wins'],
    template = context_temp)

# Creamos la cadena de LangChain con el LLM y el template
chain2 = LLMChain(llm=llm, prompt=template, output_key="Optiomal_move") # Estos dos parámetros son los más importantes para la cadena




In [123]:
# CONCATENACIÓN DE CADENAS

final_chain = SequentialChain(
    chains=[chain1,chain2],
    input_variables=['user_poker_name','player','stage','seat','play','n_plays','n_raises','n_wins'],
    output_variables=['Current_situation','Optiomal_move'],
    verbose=True
)


In [125]:
conclusion = final_chain({'user_poker_name':'pokeados',
             'player':'HER_MILK_MAN',
             'stage':'preflop',
             'seat':'UTG',
             'play':'call',
             'n_plays':33,
             'n_raises':25,
             'n_wins':2}
)



> Entering new SequentialChain chain...

> Finished chain.


In [128]:
print(conclusion['Optiomal_move'])

Based on the information provided, here is the move I would make and the reasoning behind it:

- **Move:** **Fold**
- **Why:**
  1. **HER_MILK_MAN's Aggressive Tendencies:** With HER_MILK_MAN having played 33 hands, raising 25 times, and only winning twice, it suggests a highly aggressive but possibly reckless playing style. Given that he called from UTG (a position requiring strong hands typically), his range could still be decently strong. I would prefer not to engage with a potentially strong hand from an early position unless I hold a premium hand myself.
  
  2. **Stack Preservation:** With a stack of 2,384 chips, my priority should be on identifying better opportunities to double up or accumulate chips with a stronger hand or a more advantageous position. Engaging in a pot against a potentially strong range from UTG, especially given the stack sizes of other players who may also decide to enter the pot, could put my tournament life at unnecessary risk.


In [111]:
imag = mpimg.imread(image_path)
plt.imshow(imag)
plt.axis('off')  # Ocultar los ejes
plt.show()


In [ ]:


# Ejecutamos la cadena con los datos
result = chain.run(
   { user_poker_name='pokeados',
    player='HER_MILK_MAN',
    stage='preflop',
    seat='UTG',
    play = 'call',
    n_plays = 33,
    n_raises = 25,
    n_wins = 2},
    imagen=f"data:image/png;base64,{base64_image}"
)

In [68]:
# Pasémosle el prompt al modelo
respuesta = gptmodel.invoke(prompt_test)
print(respuesta.content)

Given the information provided, there are several factors to consider before making a decision:

1. **Player Statistics:** HER_MILK_MAN has played 33 hands with 25 raises and only 2 wins. This suggests that HER_MILK_MAN is an extremely aggressive player (raising around 75% of the hands), but with a low success rate. This could indicate that they are often playing weak hands or bluffing.

2. **Position:** HER_MILK_MAN is calling from the UTG (Under The Gun) position, which is typically a strong position, as players are usually more cautious and play stronger hands from here. However, given their aggressive and loose statistics, this call may not necessarily indicate strength.

3. **Your Position and Hand Strength:** Your decision should also be influenced by your position (are you in early, middle, or late position?) and the strength of your hand. Without specific information on your hand, I can provide a general strategy.

### Potential Moves:

- **Raise:** If you have a strong hand (e

In [30]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import base64

# Función para codificar la imagen
def encode_image(image_path):
    with open(image_path, 'rb') as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

# Imagen
image_path = f"{IMG_DIR}/Mano_2024-05-12:23-14-41.png"
base64_image = encode_image(image_path)

# Define la plantilla del prompt para describir la imagen
prompt_text = """Extract all relevant information from the table, without ambiguities, to pass to the next chain.
Image data: {image_data}"""

# Crear el template
template = PromptTemplate(
    input_variables=['image_data'],
    template=prompt_text
)

# Crear la cadena de LangChain para describir la imagen
llm = ChatOpenAI(model_name="gpt-4o", openai_api_key=POKER_GPT_KEY)
chain1 = LLMChain(llm=llm, prompt=template)

# Ejecutar la cadena para obtener la descripción de la imagen
description = chain1.run(image_data=f"data:image/jpeg;base64,{base64_image}")

print(description)


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-vOpN9RsS0uOfo4bglanlWYYS on tokens per min (TPM): Limit 30000, Requested 146256. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}